<center><img src=img/MScAI_brand.png width=70%></center>

# Memoization

"Those who cannot remember the past are doomed to repeat it" 

-- George Santayana, quoted by https://blog.usejournal.com/top-50-dynamic-programming-practice-problems-4208fed71aa3

Memoization (not memo**R**ization): if a function

* may be called often with the same arguments, and 
* is deterministic, and 
* has no side effects, and
* this is enough to make our program slow; then 

it might be better to *store* the result the first time it's calculated, and in future save time by just returning that result.

It was called a machine learning technique when it was invented, and according to some definitions, it is.

In this notebook, we'll motivate the idea with an example, and then we'll implement the idea for understanding. Then we'll see how to take advantage of Python's own implementation instead.

Recall the Fibonacci numbers:

In [24]:
def fibonacci(n):
    if n in (0, 1): 
        return n
    else: 
        return (fibonacci(n - 1) + 
                fibonacci(n - 2))

In [25]:
fibonacci(10)

55

For small $n$, this is fine. But for even slightly larger $n$, it becomes slow!

In [28]:
# this % magic works in IPython 
# or Jupyter Notebook
%timeit fibonacci(35) 

2.84 s ± 6.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


A lot of time is being wasted here. Let's see:

In [29]:
def fibonacci(n):
    print(f"Calculating fibonacci({n})")
    if n in (0, 1): 
        return n
    else: 
        return (fibonacci(n - 1) + 
                fibonacci(n - 2))

In [30]:
fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(1)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(1)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(1)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(1)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)
Calculating fibonacci(3)

55

Look at that: we are re-computing `fibonacci(0)`, `fibonacci(1)`, etc., over and over! Why does that happen? Well, look at the **call graph**:

<center><img src="img/fibonacci-call-graph.png" width=45%></center>

<font size=1>From Downey, *Think Python*</font>

The solution to this is memoization. Very simply, we create a *cache* of known results, and consult it before running the function. We'll do it in a simple but Fibonacci-specific way first, and then see a way to generalise it.

In [1]:
# From Downey, Think Python
cache = {0: 0, 1: 1} # F0 and F1
def fibonacci_mem(n):
    if n in cache:
        return cache[n]
    print(f"Calculating fibonacci_mem({n})")
    cache[n] = fibonacci_mem(n - 1) + fibonacci_mem(n - 2)
    return cache[n]

In [2]:
fibonacci_mem(10)

Calculating fibonacci_mem(10)
Calculating fibonacci_mem(9)
Calculating fibonacci_mem(8)
Calculating fibonacci_mem(7)
Calculating fibonacci_mem(6)
Calculating fibonacci_mem(5)
Calculating fibonacci_mem(4)
Calculating fibonacci_mem(3)
Calculating fibonacci_mem(2)


55

The above is great and it works. The same technique can be used for other functions. 

But with this approach, for each function we want to memoize, we have to go in and edit that function. It would be nicer if we could write the memoization idea just once, and then (*lego-like*) combine that with any existing function. Well, we can!

In [3]:
def fibonacci(n):
    print(f"Calculating fibonacci({n})")
    if n in (0, 1): 
        return n
    else: 
        return (fibonacci(n - 1) + 
                fibonacci(n - 2))

Here is the higher-order trick which allows us to memoize any function `f`: we define one fn inside another.

In [9]:
def memoize(f):
    cache = {}
    def mem_f(*args): 
        if args in cache: # "cache hit"
            return cache[args]
        else: # "cache miss"
            cache[args] = f(*args)
            return cache[args]
    return mem_f

In [5]:
# memoize our *original* fibonacci function
fibonacci = memoize(fibonacci) 
fibonacci(10)        

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

`memoize` is a higher-order function: it takes a function as an argument and returns another function.

Notice that above, we don't assume that the arguments to `f` will always be just `n`. They could be anything. So we capture the arguments in `*args` and then pass that when calling `f` inside `mem_f`.

### Closures

* When we try to use `mem_f` elsewhere, `memoize` doesn't seem to exist anymore! It has finished! Shouldn't its namespace be deleted? How can `mem_f` access `cache`?

By the way, above the function `mem_f` was defined *inside* another function `memoize` (and then it was returned). That's ok. But an interesting question arises. `mem_f` has to access some data -- `cache` -- which was also defined inside `memoize`. We expect it to have access to `cache` because the environment model says we can accesss names like `cache` in the enclosing namespace, that of `memoize`. But when we return and try to use `mem_f` elsewhere, `memoize` doesn't seem to exist anymore! It has finished! Shouldn't its namespace be deleted?

Well, this is a special case: Python knows that this will happen, so it keeps the necessary data alive in a special data structure which goes everywhere with `mem_f`. Together, they are called a *closure*.

### Decorators

The pattern
```python
fibonacci = memoize(fibonacci)
```
is a common one with HOFs which take in one function argument and return some variant of it. There is a special *decorator* syntax which basically does that for us:

In [6]:
@memoize
def fibonacci(n):
    if n in (0, 1): 
        return n
    else: 
        return (fibonacci(n - 1) + 
                fibonacci(n - 2))

In [7]:
fibonacci(10)

55

Memoization is a useful trick, so it's not a surprise that it's provided as a decorator in the standard library. It's usually better to use this than to write it ourselves.

In [3]:
import functools
@functools.lru_cache(maxsize=100)
def fibonacci(n):
    if n in (0, 1): 
        return n
    else: 
        return (fibonacci(n - 1) + 
                fibonacci(n - 2))

### Arguments must be hashable

We are going to put `*args` into a `dict`, so the arguments must be **hashable**. If not, it fails!

In [8]:
@functools.lru_cache(maxsize=100)
def f(L): return len(L)
f([4, 5, 6])

TypeError: unhashable type: 'list'

### Cache replacement

* `maxsize` versus unlimited
* LRU, FIFO, LFU
* RAM v CPU trade-off

In our `memoize`, the cache was unlimited in size. In some situations it could grow very big and steal all our RAM.

In `functools.lru_cache`, we can control how large the cache is allowed to become using `maxsize`. Use `maxsize=None` for no limit on size.

It is a *least recently used* (LRU) cache, meaning that when it exceeds `maxsize`, it is the *oldest* entries which are discarded. (Other cache replacement schemes include *first-in first-out* (FIFO) and *least-frequently used* (LFU).) With a limited cache we can control the trade off of memory usage against CPU usage.